# Fully Connected MLP ANN Powered By Simple Genetic Algorithm

A co-worker dropped aneat article the other day, and it brought me back to the good ol' days in Evolutionary Computation. Back when Steven-1 sat by my side and we laughed, learned, and did stuff to keep us awake. Let's implement it and see where we land after a trip down memory lane.

## Genetic Agorithm Basic Flow

1. Start with a random population and score them all with a single, or more, fitness functions.
2. Select *n* best performers and *m* number of randomly selected *not best performers* (mitigate local max trap), and kill the rest.
3. Breed the best and bring this next generation back up to the original population size with new children.
4. Mutate some percentage of the next generation with some scheme (check the old class file for some neat schemes from Dr. Wainright).
5. Rinse and repeat until stop condition is achieved.

### Applying this to the ANN

As per the article, we will perform the image classifying with a fully connected MLP ANN. The ANN will have the following properties adjusted by the genetic algorithm:

+ number of layers
+ number of neurons per layer
+ dense layer activation function
+ network optimizer

The steps to employing the gentic algorithm with this network are as follows:

1. Initialize *N* random networks to create our population.
2. Score each network. This takes some time: We have to train the weights of each network and then see how well it performs at classifying the test set. Since this will be an image classification task, we’ll use classification accuracy as our fitness function.
3. Sort all the networks in our population by score (accuracy). We’ll keep some percentage of the top networks to become part of the next generation and to breed children.
4. Rrandomly keep a few of the non-top networks. This helps find potentially lucky combinations between worse-performers and top performers, and also helps keep us from getting stuck in a local maximum.
5. Now that we’ve decided which networks to keep, we randomly mutate some of the parameters on some of the networks.
6. Here comes the fun part: Let’s say we started with a population of 20 networks, we kept the top 25% (5 nets), randomly kept 3 more loser networks, and mutated a few of them. We let the other 12 networks die. In an effort to keep our population at 20 networks, we need to fill 12 open spots. It’s time to breed!

Not my words... but, better than my own.

## Defining the Functions

We'll start off with making some functions to help us get this wagon train rolling.

Create the population, seeding it with a number of networks with random settings.

In [1]:
def create_population(self, count):
    """Create a population of random networks.
    Args:
        count (int): Number of networks to generate, aka the
            size of the population
    """
    pop = []
    for _ in range(0, count):
        # Create a random network.
        network = Network(self.nn_param_choices)
        network.create_random()

        # Add the network to our population.
        pop.append(network)

    return pop

Breed the children of the best performing parents, differing each generation. The method employed here is randomly chosen parameters from mother and father to be passed to the child.

In [2]:
def breed(self, mother, father):
    """Make two children as parts of their parents.
    Args:
        mother (dict): Network parameters
        father (dict): Network parameters
    """
    children = []
    for _ in range(2):

        child = {}

        # Loop through the parameters and pick params for the kid.
        for param in self.nn_param_choices:
            child[param] = random.choice(
                [mother.network[param], father.network[param]]
            )

        # Now create a network object.
        network = Network(self.nn_param_choices)
        network.create_set(child)

        children.append(network)
        
        return children

Mutate using a random parameter selection to be mjutated and randomly change it. Again, there are countless number of mutation schemes, but this is what was used in the article.

In [3]:
def mutate(self, network):
    """Randomly mutate one part of the network.
    Args:
        network (dict): The network parameters to mutate
    """
    # Choose a random key.
    mutation = random.choice(list(self.nn_param_choices.keys()))

    # Mutate one of the params.
    network.network[mutation] = random.choice(self.nn_param_choices[mutation])

    return network

And now the beefcake: Evolve the population by a single generation. Run this for as many generations you want.

In [4]:
def evolve(self, pop):
    """Evolve a population of networks.
    Args:
        pop (list): A list of network parameters
    """
    # Get scores for each network.
    graded = [(self.fitness(network), network) for network in pop]

    # Sort on the scores.
    graded = [x[1] for x in sorted(graded, key=lambda x: x[0], reverse=True)]

    # Get the number we want to keep for the next gen.
    retain_length = int(len(graded)*self.retain)

    # The parents are every network we want to keep.
    parents = graded[:retain_length]

    # For those we aren't keeping, randomly keep some anyway.
    for individual in graded[retain_length:]:
        if self.random_select > random.random():
            parents.append(individual)

    # Randomly mutate some of the networks we're keeping.
    for individual in parents:
        if self.mutate_chance > random.random():
            individual = self.mutate(individual)

    # Now find out how many spots we have left to fill.
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []

    # Add children, which are bred from two remaining networks.
    while len(children) < desired_length:

        # Get a random mom and dad.
        male = random.randint(0, parents_length-1)
        female = random.randint(0, parents_length-1)

        # Assuming they aren't the same network...
        if male != female:
            male = parents[male]
            female = parents[female]

            # Breed them.
            babies = self.breed(male, female)

            # Add the children one at a time.
            for baby in babies:
                # Don't grow larger than desired length.
                if len(children) < desired_length:
                    children.append(baby)

    parents.extend(children)

    return parents

Slap this baby togther and run it!